# 트리의 앙상블

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-3.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

# 랜덤포레스트
##RandomForestClassifier
- 전체 특성 개수의 제곱근만큼의 특성 랜덤하게 선택하여 사용
- 분류일 때는 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측함

##RandomForestRegressor
- 전체 특성 사용
- 사이킷런의 랜덤포레스트는 기본적으로 100개(default)의 결정 트리 훈련
- -> GridSearchCV, RandomSearch를 통해 적절한 하이퍼파라미터 탐색
- 회귀일 때는 단순히 각 트리의 예측을 평균함

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
#numpy, pandas, train_test_split import

wine = pd.read_csv('https://bit.ly/wine_csv_data')
#웹 상에 있는 데이터를 데이터프레임화 하여 wine에 저장

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
#alcohol, sugar, pH열을 data에 저장
target = wine['class'].to_numpy()
#class 열을 target에 저장

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)
#훈련 세트와 테스트 세트를 8:2로 나

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
#cross_validate, RandomForestClassifier 클래스 import

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
#RandomForestClassifier 객체 rf 생성
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
#cross_validate()에 rf, train_input, train_target을 인소로 전달
#n_jobs=-1로 하여 모든 CPU 코어를 사용하고, 최대한 병렬로 교차 검증 수행하도록
#return_train_score=True 를 통해 검증 점수뿐만 아니라 훈련 세트에 대한 점수도 같이 반환

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#훈련 세트와 테스트 세트 점수의 평균 출력

0.9973541965122431 0.8905151032797809


In [ ]:
rf.fit(train_input, train_target)
#rf 훈련

print(rf.feature_importances_)
#특성 중요도 출력
#dT : alcohol 0.12, sugar 0.86, pH 0.007 -> 결정트리의 중요도와 다른 결과
#랜덤포레스트가 특성의 일부를 랜덤하게 선택하여 결정트리를 훈련하기 때문에
#하나의 특성에 과도하게 집중하지 않고, Feature 중요도가 덜 치우쳐져 좀 더 많은 특성 훈련에 기여함
#overfitting 줄이고, 성능 높이는데 도움

[0.23167441 0.50039841 0.26792718]


In [ ]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
#랜덤포레스트는 자체 model 평가 기능 갖추고 있음
# -> oob : out of bags, 훈련에서 사용하지 않은 sample
#oob_score=True를 통해 부트스트랩 샘플에 포함되지 않고 남은 샘플로 결정 트리 평가

rf.fit(train_input, train_target)
#rf 훈련

print(rf.oob_score_)
#각 졀정 트리의 OOB 점수를 평균하여 출력
#oob 점수를 사용하면 교차 검증을 대신할 수 있어서 훈련 세트에 더 많은 샘플을 사용할 수 O

0.8934000384837406


## 엑스트라트리
- 랜덤포레스트와 비슷하게 동작
- 부트스트랩 샘플 사용하지 않고, 각 결정트리 만들 때 전체 훈련 세트를 사용함 -> 속도 느려지지만 성능 높음
- 노드 무작위로 분할(dT에서 splitter='random' 로 사용할 수 있으나, ExtraTreesClassifier 클래스 사용) -> 빠른 계산 속도
- 엑스트라 트리는 무작위성이 좀 더 크기 때문에 랜덤포레스트보다 더 많은 결정트리를 훈련해야 함(랜덤포레스트 결정트리 default 100)
- -> GridSearchCV, RandomSearch 통해서 적절한 파라미터 찾기

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
#ExtraTreesClassifier 클래스 import

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
#ExtraTreesClassifier 의 객체 et 생성
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
#cross_validate()에 et, train_input, train_target을 인수로 전달
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#훈련 세트와 테스트 세트 점수 출력

0.9974503966084433 0.8887848893166506


In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_)
#et의 특성 중요도 출력

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅
- 깊이가 얕은 결정 트리를 사용하여 이전 트리의 잔차(표본집단에서 예측과 관측값의 차이)를 보완하는 방식으로 앙상블하는 방법
- GradientBoostingClassifier는 기본적으로 깊이가 3인 결정 트리 100개 사용
- overfitting에 강하고, 높은 일반화 성능 기대 가능
- 경사하강법으로 트리를 앙상블에 추가
- 분류에서는 로지스틱 손실 함수 사용
- 회귀에서는 평균 제곱 오차 함수 사용
- 잔차를 활용하여 오차 함수 사용

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
#GradientBoostingClassifier 클래스 import

gb = GradientBoostingClassifier(random_state=42)
#GradientBoostingClassifier 객체 gd 생성
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
#cross_validate()(교차검증)에 gb, train_ipnut, train_target 인수로 전달

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#훈련 세트와 테스트 세트 점수 출력
#overfitting 발생 X <- 깊이가 얕은 결정트리 사용하기 때문

0.8881086892152563 0.8720430147331015


In [ ]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
#성능 높이기 위해 n_estimators=500으로 하여 결정 트리 개수를 늘림
#default 값 : n_estimators=100, (학습률 learning rate)lr=0.1
#GridSearchCV, RandomSearch로 적절한 하이퍼 파라미터 찾는 것이 효율적

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#훈련 세트와 테스트 세트 점수 출력

0.9464595437171814 0.8780082549788999


In [ ]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)
#gb 특성 중요도 출력
#그레디언트 부스팅이 랜덤포레스트보다 일부 특성(당도)에 더 집중(잔차 보완에 중심을 맞춰서 중요한 feature에 focusing)
#랜덤포레스트보다 조금 더 높은 성능
#but, 순서대로 트리를 추가하기 때문에 훈련 속도 느림
#subsample 매개변수 : 기본값은 1.0으로 전체 훈련 세트(gd)를 사용하지만,
#1보다 작으면 훈련 세트의 일부를 사용해 확률적 경사 하강법(sgd), 미니배치 경사 하강법(mini-batch) 사용하여 속도 빠르게

[0.15872278 0.68010884 0.16116839]


## 히스토그램 기반 부스팅

In [ ]:
# 사이킷런 1.0 버전 아래에서는 다음 라인의 주석을 해제하고 실행하세요.
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [ ]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [ ]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [ ]:
hgb.score(test_input, test_target)

0.8723076923076923

#### XGBoost
- 히스토그램 기반 gradient boosting 알고리즘 라이브러리 (묶음 안에서 best split을 찾기 때문에 히스토그램 기반이라고 함)
- feature를 256개의 구간으로 분할하여, 빠르게 최적의 분할을 찾음

In [ ]:
from xgboost import XGBClassifier
#XGBClassifier 클래스 import

xgb = XGBClassifier(tree_method='hist', random_state=42)
#XGBClassifier의 객체 xgb 생성
# tree_method(트리 빌딩 방법) : split point를 잡을 때, 히스토그램 사용 여부
# auto, exact, hist, gpu_hist, approx -> 다양한 파라미터 => GridSearhCV, RandomSearch 활용~~
# 데이터가 클수록 분할하는 것이 유리하나, 데이터가 작을 때는 오히려 단점이 될 수도
#auto 데이터 크기에 따라 자동으로 조절 / exact 전수조사(작은 데이터) / hist 구간 나눠서
#gpu_hist 로컬에 gpu가 있다면 좀 더 빠르게 훈련 / approx 가중치 고려하는 hist와 비슷한 방법
#hist, gpu_hist 일반적으로 많이 사용

scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)
#cross_validate()에 xgb, train_input, train_target을 인수로 전달

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#훈련 세트와 테스트 세트 점수 출력

0.9555033709953124 0.8799326275264677


#### LightGBM
- 1) leaf wise method : 한 쪽 방향으로 성장
- 2) 연관있는 feature combine -> light해서 빠름
- MS에서 만든 히스토그램 기반 그레디언트 부스팅 라이브러리
- 다양한 파라미터 => GridSearhCV, RandomSearch 활용~~

In [ ]:
from lightgbm import LGBMClassifier
#LGBMClassifier 클래스 import
lgb = LGBMClassifier(random_state=42)
#LGBMClassifier 객체 lgb 생성
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
#cross_validate()에 lgb, train_input, train_target을 인수로 전달

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#훈련 세트와 테스트 세트 점수 출력

0.935828414851749 0.8801251203079884
